In [2]:
import pandas as pd
import json
data_type = "train"
# temporal
with open("/ssd3/ming/SMPD2024/CPDN/dataset/%s_allmetadata_json/%s_temporalspatial_information.json" % (data_type, data_type)) as f:
    train_ts = json.load(f)
    vuid_arr = []
    time_arr = []
    lon_arr, lat_arr, geoacc_arr = [], [], []
    ts_dict = {}
    # 对时间排序
    for k, v in enumerate(train_ts):
        vuid = v['Uid'] + '/' + v['Pid']
        vuid_arr.append(vuid)
        time_arr.append(v['Postdate'])
        if v['Geoaccuracy'] == '0':
            lon_arr.append(0)
            lat_arr.append(0)
            geoacc_arr.append(0)
        else:
            lon_arr.append(v['Longitude'])
            lat_arr.append(v['Latitude'])
            geoacc_arr.append(v['Geoaccuracy'])
    id_arr = list(range(0, len(time_arr)))
    time_sort, id_sort = (list(t) for t in zip(*sorted(zip(time_arr, id_arr))))
    vuid_sort = []
    lon_sort, lat_sort, geoacc_sort = [], [], []
    for id in id_sort:
        vuid_sort.append(vuid_arr[id])
        lon_sort.append(lon_arr[id])
        lat_sort.append(lat_arr[id])
        geoacc_sort.append(geoacc_arr[id])
f.close()
with open("/ssd3/ming/SMPD2024/CPDN/dataset/%s_allmetadata_json/%s_additional_information.json" % (data_type, data_type)) as f:
    train_add = json.load(f)
    public_arr, public_sort, path_arr, path_sort = [], [], [], []
    for v in train_add:
        vuid = v['Uid'] + '/' + v['Pid']
        public_arr.append(v['Ispublic'])
        path_arr.append(v['Pathalias'])
    for id in id_sort:
        public_sort.append(public_arr[id])
        path_sort.append(path_arr[id])
f.close()


In [3]:
import re
import requests
import time
from requests.adapters import HTTPAdapter
s = requests.session()
s.keep_alive = False
s.mount('https://', HTTPAdapter(max_retries=3))
#爬取数据
user_path_dict = {}
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36 Edg/97.0.1072.55"
}
with open('/ssd3/ming/SMPD2024/CPDN/dataset/train_data_add.csv', 'a+') as f:
    f.writelines("vuid, followers, following, totalviews, totaltags, totalgeotags, totalfaves, totalingroup, true_id") 
    df = pd.DataFrame(columns=['id', 'followers', 'following', 'totalviews', 'totaltags', 'totalgeotags', 'totalfaves', 'totalingroup', 'true_id'])
    for i, path in enumerate(path_sort):
        flag = False
        if i < 402:
            continue
        uid = vuid_sort[i].split('/')[0]
        print(i, vuid_sort[i])
        if user_path_dict.get(uid):
            if user_path_dict.get(uid) != 'none':
                data = user_path_dict.get(uid)
            else:
                data = [vuid_sort[i], 0, 0, 0, 0, 0, 0, 0, 0]
        else:
            if path != 'None':
                url = 'https://www.flickr.com/people/' + path
            else:
                url = None
            if url is not None:
                while flag is not True:
                    try:
                        resp = requests.get(url, headers=headers, timeout=8)
                        if resp.status_code == 200:
                            result = resp.text
                            p1 = '>.*Follower'
                            p2 = '>[0-9A-Z\s\.]+Following'
                            p3 = 'totalViews.*?}'
                            follower = re.findall(p1, result)[0].split(' ')[0][1:]
                            following = re.findall(p2, result)[0].split(' ')[0][1:]
                            others = re.findall(p3, result)[0].split(',')
                            
                            print(others)
                            
                            totalviews = others[0].split(':')[1]
                            totaltags = others[1].split(':')[1]
                            totalgeotags = others[2].split(':')[1]
                            totalfaves = others[3].split(':')[1]
                            totalingroup = others[4].split(':')[1]
                            true_id = others[5].split(':')[1][1:-2]
                            data = [vuid_sort[i], follower, following, totalviews, totaltags, totalgeotags, totalfaves, totalingroup, true_id]
                            
                            # followers = html.xpath()
                            # print(html)  # 拿到页面源代码
                            resp.close()  # 关掉resp
                            user_path_dict[uid] = data
                        else:
                            user_path_dict[uid] = 'none'
                            data = [vuid_sort[i], 0, 0, 0, 0, 0, 0, 0, 0]
                        flag = True
                    except requests.exceptions.RequestException as e:
                        print(e)
                        flag = False
                        time.sleep(5)
            else:
                user_path_dict[uid] = 'none'
                data = [vuid_sort[i], 0, 0, 0, 0, 0, 0, 0, 0]
        f.writelines('\n' + ','.join(str(s) for s in data))
        f.flush()


402 37745@N71/294014
['totalViews":17409', '"totalTags":231', '"totalGeotagged":0', '"totalFaves":22', '"totalInGroup":0', '"id":"136959387@N04"}']
403 104561@N58/1035815
['totalViews":41909', '"totalTags":208', '"totalGeotagged":0', '"totalFaves":57', '"totalInGroup":1', '"id":"107488033@N07"}']
404 37745@N71/296351
405 37745@N71/296346
406 37745@N71/275387
407 37745@N71/286720
408 37745@N71/294015
409 37745@N71/294016
410 16984@N15/305534
['totalViews":1121790', '"totalTags":7670', '"totalGeotagged":1932', '"totalFaves":1432', '"totalInGroup":594', '"id":"8940551@N05"}']
411 16984@N15/305535
412 35342@N41/272862
413 23935@N29/157552
['totalViews":39513', '"totalTags":358', '"totalGeotagged":27', '"totalFaves":227', '"totalInGroup":114', '"id":"93785176@N06"}']
414 24470@N54/160982
['totalViews":23487', '"totalTags":149', '"totalGeotagged":0', '"totalFaves":44', '"totalInGroup":60', '"id":"75724224@N05"}']
415 21435@N15/155763
416 9914@N58/159743
['totalViews":4772166', '"totalTags":5

KeyboardInterrupt: 